# Alibaba Challenge

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, losses

AUTOTUNE = tf.data.experimental.AUTOTUNE
np.set_printoptions(precision=3, suppress=True)

tf.__version__

### Analyze Data

In [ ]:
df = pd.read_csv('images/train_data.txt', skipinitialspace=True)
df

In [ ]:
df.values.shape, df.values

In [ ]:
df['is_ellipse'].astype(np.float32).plot.hist(title='Ellipses Distribution');

In [ ]:
df[['center_x', 'center_y']].plot.hist(bins=50, alpha=0.5, title='Centers Distribution');

In [ ]:
df['angle'].plot.hist(title='Angles Distribution');

In [ ]:
df[['axis_1', 'axis_1']].plot.hist(bins=50, alpha=0.5, title='Axis Distribution');

### Build Data

In [ ]:
def make_dataset(path):
    df = pd.read_csv(path, skipinitialspace=True)
    images = df.pop('images')
    df = df.astype(np.float32)
    ds = tf.data.Dataset.from_tensor_slices((images.values, df.to_dict('list')))
    return ds

In [ ]:
train_dataset = make_dataset('images/train_data.txt')
test_dataset = make_dataset('images/test_data.txt')

In [ ]:
for path, d in train_dataset.take(5):
    print(f'path={path}: is_ellipse={d["is_ellipse"]}, center_x={d["center_x"]}, center_y={d["center_y"]}, angle={d["angle"]}, axis_1={d["axis_1"]}, axis_2={d["axis_2"]}')

In [ ]:
for path, d in test_dataset.take(5):
    print(f'path={path}: is_ellipse={d["is_ellipse"]}, center_x={d["center_x"]}, center_y={d["center_y"]}, angle={d["angle"]}, axis_1={d["axis_1"]}, axis_2={d["axis_2"]}')

In [ ]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img


def preprocess(path, d):
    X = load_img(path)
    y = d['is_ellipse']
    return X, y


def display_data(X, y):
    plt.figure(figsize=(10,10))
    for i in range(16):
        plt.subplot(4,4,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(X[i], cmap=plt.cm.binary)
        plt.xlabel(f'{y[i]:.3f}')
    plt.show()    

In [ ]:
train_ds = train_dataset.shuffle(10000).map(preprocess, num_parallel_calls=AUTOTUNE, deterministic=False).batch(16).prefetch(AUTOTUNE)
test_ds = test_dataset.map(preprocess, num_parallel_calls=AUTOTUNE, deterministic=False).batch(16).prefetch(AUTOTUNE)

### Verify Data

In [ ]:
for X, y in train_ds.take(1):
    display_data(X, y)

In [ ]:
for X, y in test_ds.take(1):
    display_data(X, y)

### Simple Model

In [ ]:
model = models.Sequential()
model.add(layers.Input(shape=(50, 50, 3)))
model.add(layers.Lambda(lambda x: x/255))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=['accuracy'])

### Train

In [ ]:
history = model.fit(train_ds, validation_data=test_ds, epochs=15)

### Train Model

In [ ]:
def preprocess(path, d):
    X = load_img(path)
    is_ellipse = d['is_ellipse']
    center_x = d["center_x"]
    center_y = d["center_y"]
    angle = d["angle"]
    axis_1 = d["axis_1"]
    axis_2 = d["axis_2"]
    angle = tf.where(angle >= 180, 360-angle, angle)
    angle = tf.where(angle == 180, 0., angle)
    angle = tf.where(tf.abs(axis_1-axis_2) == 0, 0., angle)
    angle = tf.one_hot(tf.cast(angle, tf.int32), 179)
    center_x /= 50
    center_y /= 50
    axis_1 /= 25
    axis_2 /= 25
    y = tf.stack([is_ellipse, center_x, center_y, axis_1, axis_2], axis=-1)
    y = tf.concat([y, angle], axis=-1)
    X = tf.cast(X, tf.float32) / 127.5 - 1.0
    return X, y

In [ ]:
train_ds = train_dataset.shuffle(10000).map(preprocess, num_parallel_calls=AUTOTUNE, deterministic=False).batch(128).prefetch(AUTOTUNE)
test_ds = test_dataset.map(preprocess, num_parallel_calls=AUTOTUNE, deterministic=False).batch(16).prefetch(AUTOTUNE)

In [ ]:
for X, y in test_ds.take(1):
    print(X.shape, y.shape)
    print(y[:10])

### Custom Loss Function

In [ ]:
def custom_loss(y_true, y_pred):
    is_ellipse_loss = losses.binary_crossentropy(y_true[...,0:1], y_pred[...,0:1])
    is_ellipse_loss = tf.reduce_mean(is_ellipse_loss)
    
    is_ellipse = y_true[...,0]
    
    centers_loss = losses.logcosh(y_true[...,1:3], y_pred[...,1:3])*is_ellipse
    centers_loss = 10.0 * tf.reduce_mean(centers_loss)        
    
    axis_loss = losses.logcosh(y_true[...,3:5], y_pred[...,3:5])*is_ellipse
    axis_loss = 10.0 * tf.reduce_mean(axis_loss)            
    
    #angle_loss = losses.categorical_crossentropy(y_true[...,6:], y_pred[...,6:])*is_ellipse
    angle_loss = tf.abs(tf.argmax(y_true[...,6:])-tf.argmax(y_pred[...,6:]))
    angle_loss = tf.cast(angle_loss, tf.float32)
    angle_loss = 1./ 180 * tf.reduce_mean(angle_loss)
    
    #tf.print(is_ellipse_loss, angle_loss, centers_loss, axis_loss)

    return is_ellipse_loss + angle_loss + centers_loss + axis_loss

### Model

In [ ]:
# x = layers.Conv2D(32, (3, 3), activation='relu')(x)
# x = layers.MaxPooling2D((2, 2))(x)
# x = layers.Conv2D(64, (3, 3), activation='relu')(x)
# x = layers.MaxPooling2D((2, 2))(x)
# x = layers.Conv2D(64, (3, 3), activation='relu')(x)
# x = layers.MaxPooling2D((2, 2))(x)
# x = layers.Conv2D(128, (3, 3), activation='relu')(x)
# x = layers.Flatten()(x)

# backbone = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_shape=(50, 50, 3))
# x = backbone(x_in)
# x = layers.GlobalAveragePooling2D()(x)
# x = layers.Dense(128, activation='relu')(x)

# is_ellipse = layers.Dense(1, activation='sigmoid')(x)
# angle = layers.Dense(1, activation=None)(x)
# centers = layers.Dense(2, activation=None)(x)
# axis = layers.Dense(2, activation=None)(x)

backbone =tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(50, 50, 3))

x = backbone.output
x = layers.Flatten()(x)

is_ellipse = layers.Dense(1, activation='sigmoid')(x)
centers = layers.Dense(2, activation='sigmoid')(x)
axis = layers.Dense(2, activation='sigmoid')(x)
angle= layers.Dense(179, activation='softmax')(x)

x = tf.concat([is_ellipse, angle, centers, axis], axis=-1)

# create the new model
model = tf.keras.Model(inputs=backbone.input, outputs=x)

# model compilation
model.compile(loss=custom_loss, optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9))

#model.summary()

In [ ]:
model_cp = tf.keras.callbacks.ModelCheckpoint('checkpoints/cp.ckpt', save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True, verbose=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau()

history = model.fit(train_ds, validation_data=test_ds, epochs=100, callbacks=[model_cp, reduce_lr])

### Load Best Model

In [524]:
model.load_weights('checkpoints/cp.ckpt')
model.evaluate(test_ds)

63/63 [==============================] - 1s 10ms/step - loss: 0.7220


0.7220373153686523

### Test results

In [525]:
val_ds = test_dataset.map(preprocess).batch(1)

results = [(y[0].numpy(),model(X)[0].numpy()) for X, y in val_ds]

### Ellipse Classification Accuracy

In [526]:
from sklearn.metrics import accuracy_score

y_true = [y[0] for y,p in results]
y_pred = [p[0] > 0.5 for y,p in results]
    
score = accuracy_score(y_true, y_pred)

print(f'Accuracy: {score}')

Accuracy: 1.0


### Angle Error

In [527]:
from sklearn.metrics import mean_absolute_error

y_true = [y[1]*180 for y,p in results if y[0] == 1]
y_pred = [p[1]*180 for y,p in results if y[0] == 1]

error = mean_absolute_error(y_true, y_pred)

print(f'Error: {error}\n')

for y,p in list(zip(y_true, y_pred))[:10]:
    print(f'y={y}, p={p}')

Error: 45.21313327353202

y=118.80000472068787, p=51.0269558429718
y=90.0, p=10.213660150766373
y=100.80000042915344, p=68.95139694213867
y=100.80000042915344, p=60.931795835494995
y=64.80000257492065, p=47.31012761592865
y=57.59999871253967, p=48.16783368587494
y=61.200000643730164, p=0.15837416402064264
y=115.19999742507935, p=79.83592987060547
y=82.80000150203705, p=58.416441679000854
y=93.59999656677246, p=22.52397358417511


### Centers Error

In [528]:
from sklearn.metrics import mean_squared_error

y_true = [y[2:4]*50 for y,p in results if y[0] == 1]
y_pred = [np.round(p[2:4]*50) for y,p in results if y[0] == 1]

error = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'Error: {error}\n')

for y,p in list(zip(y_true, y_pred))[:10]:
    print(f'y={y}, p={p}')

Error: 15.733471870422363

y=[30. 26.], p=[10. 12.]
y=[31. 40.], p=[10. 17.]
y=[29. 12.], p=[19.  7.]
y=[24. 14.], p=[14.  5.]
y=[24. 18.], p=[20.  9.]
y=[16. 18.], p=[13. 10.]
y=[28. 46.], p=[ 6. 19.]
y=[24. 30.], p=[14. 10.]
y=[30.  8.], p=[25.  5.]
y=[31. 30.], p=[13. 16.]


### Axis Error

In [529]:
from sklearn.metrics import mean_squared_error

y_true = [y[4:6]*25 for y,p in results if y[0] == 1]
y_pred = [np.abs(p[4:6])*25 for y,p in results if y[0] == 1]

error = np.sqrt(mean_squared_error(y_true, y_pred))

print(f'Error: {error}\n')

for y,p in list(zip(y_true, y_pred))[:10]:
    print(f'{y}, {p}')

Error: 7.824385643005371

[24.  0.], [6.503 0.   ]
[20. 25.], [10.132  0.   ]
[10.  0.], [2.649 0.   ]
[13.  0.], [7.215 0.   ]
[3. 0.], [3.96 0.  ]
[5. 0.], [6.766 0.   ]
[23. 25.], [12.748  0.   ]
[3. 0.], [1.849 0.   ]
[7. 0.], [1.723 0.   ]
[22.  0.], [7.296 0.   ]
